In [16]:
import os
from glob import glob
from twelvelabs import TwelveLabs
from twelvelabs.models.task import Task

# The path to the directory containing the videos you wish to upload.
VIDEO_PATH = "./videos/*.mp4" # Example: "/videos/*.mp4
API_KEY = os.environ['API_KEY']
INDEX_NAME = "test_index-2vyufvcygiucvfdtrxdytcfvygjbhujknjilnjiniu2"

In [17]:
client = TwelveLabs(api_key=API_KEY)

index = client.index.create(
    name=INDEX_NAME,
    engines=[
        {
            "name": "pegasus1",
            "options": ["visual", "conversation"],
        }
    ]
)
print(f"Created index: id={index.id} name={index.name} engines={index.engines}")

Created index: id=666fc08a0090a8f99febc3ec name=test_index-2vyufvcygiucvfdtrxdytcfvygjbhujknjilnjiniu2 engines=root=[Engine(name='pegasus1', options=['visual', 'conversation'], addons=None)]


In [8]:
video_files = glob(VIDEO_PATH)
for video_file in video_files:
  print(f"Uploading {video_file}")
  task = client.task.create(index_id=index.id, file=video_file, language="en")
  print(f"Created task: id={task.id}")

  # (Optional) Monitor the video indexing process
  # Utility function to print the status of a video indexing task
  def on_task_update(task: Task):
          print(f"  Status={task.status}")
  task.wait_for_done(sleep_interval=50, callback=on_task_update)
  if task.status != "ready":
      raise RuntimeError(f"Indexing failed with status {task.status}")
  print(f"Uploaded {video_file}. The unique identifer of your video is {task.video_id}.")

Uploading ./videos/Fei_Fei_Li_AI_Summit_2024.mp4
Created task: id=666fa3ca0090a8f99febc30c
  Status=pending
  Status=pending
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=ready
Uploaded ./videos/Fei_Fei_Li_AI_Summit_2024.mp4. The unique identifer of your video is 666fa3cbd22b3a3c97bf289a.


In [9]:
videos = client.index.video.list(index.id)
for video in videos:
    print(f"Generating text for {video.id}")

    res = client.generate.gist(video_id=video.id, types=["title", "topic", "hashtag"])
    print(f"Title: {res.title}\nTopics={res.topics}\nHashtags={res.hashtags}")

    res = client.generate.summarize(video_id=video.id, type="summary")
    print(f"Summary: {res.summary}")

    print("Chapters:")
    res = client.generate.summarize(video_id=video.id, type="chapter")
    for chapter in res.chapters:
        print(
            f"  chapter_number={chapter.chapter_number} chapter_title={chapter.chapter_title} chapter_summary={chapter.chapter_summary} start={chapter.start} end={chapter.end}"
        )

    print("Highlights:")
    res = client.generate.summarize(video_id=video.id, type="highlight")
    for highlight in res.highlights:
        print(
            f"  Highlight={highlight.highlight} start={highlight.start} end={highlight.end}"
        )

    res = client.generate.text(video_id=video.id, prompt="Based on this video, I want to generate five keywords for SEO (Search Engine Optimization).")
    print(f"Open-ended Text: {res.data}")

Generating text for 666fa3cbd22b3a3c97bf289a
Title: From Darkness to Sight: A Journey of Spatial Intelligence and AI Evolution
Topics=[' Spatial Intelligence and AI Advancements']
Hashtags=['AI', 'spatial intelligence', 'robotics', 'computer vision', 'generative AI', 'neural networks', 'ambient intelligence']
Summary: In an enlightening and forward-looking presentation, a speaker delves into the profound realm of spatial intelligence and its pivotal role in advancing artificial intelligence (AI) and robotics, potentially heralding a new era of human-computer interaction. The talk, given across various prestigious platforms including FAIR 2020, DATATALK, Data Summit 2021, and NeurIPS 2020, presents a comprehensive overview of the evolution of AI, from its nascent stages to the cutting-edge developments in generative AI and robotics.
The speaker begins by tracing the origins of vision in the natural world, drawing a parallel to the inception of AI's ability to "see" and interpret the wor